# 📬 LLM-powered Email Response Automation for a Fashion Store

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

# Prerequisites

### Configure OpenAI API Key.

# Task 1. Classify emails

In [ ]:
%pip install pandas openpyxl langchain langchain-community faiss-cpu gspread tiktoken python-dotenv langchain-openai


# Import Necessary Libraries

This block imports all necessary libraries including LangChain components, pandas for data manipulation, and OpenAI tools for embedding and LLM interaction.



In [ ]:
import os
import pandas as pd
from openpyxl import Workbook
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from typing import List, Dict, Union


# API Keys and Base URL

In [ ]:
import os
# Set the API key and endpoint
os.environ["OPENAI_API_KEY"] = "Your_Key_Here"
os.environ["OPENAI_BASE_URL"] = 'Base_URL_Here'


# Define LLM Model with langchain

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0.3,
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY")
)


# ✉️ Classification Prompt: Determines if an email is a product inquiry or order

Defines a prompt template used to classify emails as either "product inquiry" or "order request" using natural language understanding.



In [ ]:
## Classifiying Emails using LLM
from langchain.prompts import ChatPromptTemplate

classification_prompt = ChatPromptTemplate.from_template("""
Classify the following email:

Email Subject: {subject}
Email Body: {body}

Classify as either:
- "product inquiry"
- "order request"

Answer only with the classification.
""")


# 🧠 Classify Email using LLM

This function sends the subject and body of an email to the LLM for classification based on the above prompt.



In [ ]:
## Classification function:

def classify_email(subject: str, body: str) -> str:
    chain = classification_prompt | llm
    response = chain.invoke({"subject": subject, "body": body})
    return response.content.strip()


# 🧑‍💼 Extract Customer Name Prompt using LLM

Sets up a prompt to extract the customer’s name from the email body using the LLM.



In [ ]:
from langchain.prompts import ChatPromptTemplate

name_extraction_prompt = ChatPromptTemplate.from_template("""
Extract the full name of the customer from the email body below.

Email Body:
{body}

Instructions:
- If a name is mentioned at the end or within the email (e.g. "Thanks, Alex", "My name is Sarah"), return the name.
- If no name is found, return: "Customer"

ONLY return the name. Do not include any other explanation or formatting.
""")


# 📇 Name Extraction Function

Uses the prompt above to extract a name; defaults to "Customer" if no name is found.



In [ ]:
def extract_customer_name_llm(body: str) -> str:
    chain = name_extraction_prompt | llm
    response = chain.invoke({"body": body})
    name = response.content.strip()

    # Basic validation
    if 1 < len(name) <= 40:
        return name.split("\n")[0].strip()
    return "Customer"


# Task 2. Process order requests

# 📦 Prompt to extract order details using LLM

Instructs the LLM to extract structured order details (product ID, name, quantity) from an email.



In [ ]:
from langchain.prompts import ChatPromptTemplate

extract_order_prompt = ChatPromptTemplate.from_template("""
Extract the product ID, product names, and quantities from the following email:

Email Body:
{body}

Available Products:
{product_list}

Instructions:
- If the customer asks for "all remaining," return the string `"all remaining"`.
- If the quantity is numeric, return it as an integer.
- Only extract if the customer explicitly mentions wanting to buy, order, or purchase the product.

Provide the output as a JSON object in the format:
[
    {{
        "product_id": "Product ID",
        "product": "Product Name",
        "quantity": 2
    }}
]

Ensure the response is ONLY valid JSON — no extra text, explanation, or markdown.
""")



# 🧾 Extract Order Details Function

Feeds the email body and product catalog into the prompt to get back structured product requests.



In [ ]:
import json

def extract_order_details(body: str):
    product_names = products_df['name'].tolist()
    product_list = "\n".join(product_names)

    chain = extract_order_prompt | llm
    response = chain.invoke({
        "body": body,
        "product_list": product_list
    })

    try:
        clean_response = response.content.strip().replace("```json", "").replace("```", "")

        order_details = json.loads(clean_response)

        return order_details

    except json.JSONDecodeError as e:
        return []


# 🔎 RAG-based Product Search

Searches the embedded vector store using semantic similarity to retrieve the most relevant product.



In [ ]:
def search_product_with_rag(query):
    docs = retriever.invoke(query)

    if docs:
        top_match = docs[0].page_content

        # Search using the 'text' column
        matched_product = products_df[
            products_df['text'].str.contains(top_match.split('.')[0], case=False, na=False)
        ]

        if not matched_product.empty:
            return matched_product
    return pd.DataFrame()



# 📈 Recommend top products if intent is vague

Recommends top-selling products when no specific product is mentioned by the customer in the email.



In [ ]:
def search_popular_products():
    query = "popular product or best-selling item"
    docs = retriever.invoke(query)

    if docs:
        top_matches = [doc.page_content for doc in docs[:3]]

        # Loosely match the products using 'text' field
        mask = products_df['text'].apply(lambda x: any(match.lower() in x.lower() for match in top_matches))
        matched_products = products_df[mask].head(3)

        if not matched_products.empty:
            return matched_products

    # Fallback: return top 3 by stock
    return products_df.sort_values(by='stock', ascending=False).head(3)


# 📦 Process Orders from Emails

Handles direct and indirect product requests:

Matches product and updates stock.

Falls back to recommendations.

Generates tailored response using LLM.

In [ ]:
def process_order(email_id, body):
    order_details = extract_order_details(body)
    results = []
    responses = []

    # If order extraction fails → Trigger recommendations
    if not order_details:
        print(f"No direct product found in the store for {email_id}. Recommending popular products...")
        recommendations = search_popular_products()

        if not recommendations.empty:
            recommended_products = []
            response_messages = []

            for _, product in recommendations.iterrows():
                stock = int(product['stock'])
                product_id = product['product_id']
                product_name = product['name']
                price = product['price']
                quantity = 1  # Assume customer wants 1 unit

                status = "created" if stock >= quantity else "out of stock"
                products_df.loc[products_df['product_id'] == product_id, 'stock'] -= quantity

                # Save recommendation to results
                result = {
                    "email_id": email_id,
                    "product_id": product_id,
                    "product": product_name,
                    "quantity": quantity,
                    "status": status
                }
                recommended_products.append(result)

                # Format recommendation response
                response_message = f"{product_name} (ID: {product_id}) - Price: ${price}, Stock: {stock}"
                response_messages.append(response_message)

            # Save response message for all recommendations
            combined_response = "\n".join(response_messages)
            responses.append({
                "email_id": email_id,
                "response": f"We recommend the following products:\n{combined_response}"
            })
            print(f"Recommendations sent for {email_id}")
            return recommended_products, responses

    # Existing direct order handling logic
    for order in order_details:
        product_id = order.get("product_id")
        product = order["product"]
        quantity = order["quantity"]

        if product_id:
            product_info = products_df.loc[products_df['product_id'] == product_id].copy()
        else:
            product_info = products_df.loc[products_df['name'].str.contains(product, case=False, na=False)].copy()

        if product_info.empty:
            product_info = search_product_with_rag(product)
            if isinstance(product_info, pd.Series):
                product_info = product_info.to_frame().T

        if not product_info.empty:
            stock = int(product_info.iloc[0]['stock'])
            product_id = product_info.iloc[0]['product_id']
            product_name = product_info.iloc[0]['name']

            if isinstance(quantity, str) and quantity.lower() == "all remaining":
                quantity = stock
            else:
                try:
                    quantity = int(quantity)
                except ValueError:
                    continue

            if stock >= quantity:
                status = "created"
                products_df.loc[products_df['product_id'] == product_id, 'stock'] -= quantity
                remaining_stock = stock - quantity
            else:
                status = "out of stock"
                remaining_stock = stock

            # Record order result
            result = {
                "email_id": email_id,
                "product_id": product_id,
                "product": product_name,
                "quantity": quantity,
                "status": status
            }
            results.append(result)

            # Generate response
            response = generate_order_response({
                "status": status,
                "product": product_name,
                "quantity": quantity,
                "stock": remaining_stock,
                "body": body
            })
            responses.append({
                "email_id": email_id,
                "response": response
            })
            print(f"Order processed for {email_id}: {product_name} - {status}")
        else:
            error = {
                "email_id": email_id,
                "product_id": product_id,
                "product": product,
                "quantity": quantity,
                "status": "product not found"
            }
            responses.append({
                "email_id": email_id,
                "response": f"Product '{product}' not found in catalog."
            })
            print(f"Product not found for {email_id}: {product}")

    return results, responses


# Order Response Email Generator

Generates a professional email response using customer name and order details via a templated prompt.



In [ ]:
order_response_prompt = ChatPromptTemplate.from_template("""
You are a customer support agent for an online fashion store. Based on the order status below, generate a professional, polite, and production-ready email for the customer.

Order Status: {status}
Product: {product}
Quantity: {quantity}
Stock Remaining: {stock}
Customer Name: {name}

Email Template:
Dear {name},

Thank you for reaching out to us.

[Insert dynamic message based on the status — e.g., order confirmed, out of stock, etc.]

If you have any questions or need assistance with anything else, feel free to reply to this email. We’re here to help.

Best regards,
Customer Support Team
Fashion Store
""")


In [ ]:
def generate_order_response(order):
    name = extract_customer_name_llm(order.get("body", "Customer"))
    chain = order_response_prompt | llm
    response = chain.invoke({**order, "name": name})
    return response.content.strip()


# Task 3. Handle product inquiry

# Product Inquiry Handling with RAG

Uses vector search to find product context and answers the inquiry using an LLM-based template.



In [ ]:
inquiry_prompt = ChatPromptTemplate.from_template("""
You are a helpful customer support agent at an online fashion store.

Use the information below to generate a professional and friendly response to the customer’s inquiry.

Inquiry: {inquiry}
Relevant Products:
{context}
Customer Name: {name}


Email Template:
Dear {name},

Thank you for your inquiry.

[Insert dynamic product answer based on RAG context — mention product name, price, stock, and seasonal suitability.]

Please let us know if you'd like to place an order or need more information. We're happy to help!

Best regards,
Customer Support Team
Fashion Store
""")


In [ ]:
def handle_inquiry(email_id, body):
    print(f"Handling inquiry for {email_id}")

    # Use RAG to retrieve relevant products
    relevant_products = retriever.get_relevant_documents(body)
    name = extract_customer_name_llm(body)

    if relevant_products:
        # Extract metadata properly from the document
        context = "\n".join([
            f"Product: {doc.metadata.get('name', 'N/A')}\n"
            f"Price: ${doc.metadata.get('price', 'N/A')}\n"
            f"Stock: {doc.metadata.get('stock', 'N/A')}\n"
            f"Category: {doc.metadata.get('category', 'N/A')}\n"
            f"Season: {doc.metadata.get('season', 'N/A')}\n"
            f"Description: {doc.metadata.get('description', 'N/A')}"
            for doc in relevant_products
        ])


        # Pass metadata into LLM prompt
        chain = inquiry_prompt | llm
        response = chain.invoke({"inquiry": body, "context": context, "name": name})

        return response.content.strip()

    else:
        # Fallback if no product is found
        return "Sorry, we couldn't find details for your inquiry."


# Save Output to Excel

Writes processed classification results, order status, responses, and inquiries to an Excel workbook for reporting.



In [ ]:
def save_to_excel():
    with pd.ExcelWriter("Solving Business Problems with AI - Output.xlsx") as writer:
        # Save classification results
        emails_df[['email_id', 'subject', 'body', 'classification']].to_excel(
            writer, sheet_name="email-classification", index=False
        )

        # Save order processing status (with product ID)
        if order_results:
            pd.DataFrame(order_results).to_excel(
                writer, sheet_name="order-status", index=False
            )

        # Save order responses
        if order_responses:
            pd.DataFrame(order_responses).to_excel(
                writer, sheet_name="order-response", index=False
            )

        # Save inquiry responses
        if inquiry_responses:
            pd.DataFrame(inquiry_responses).to_excel(
                writer, sheet_name="inquiry-response", index=False
            )

    print("Output saved to output.xlsx")


# Load Data from Google Sheets


In [ ]:
## Loading Data From Google Sheets:

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

# Google Sheet document ID
document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'

# Read products and emails sheets
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')
emails_df.rename(columns={'message': 'body'}, inplace=True)
# Create 'text' column by combining key fields
# Embed both product description and metadata into the vector store

products_df['text'] = products_df.apply(
    lambda row: f"{row['name']}. {row['category']}. {row['description']}. "
                f"Stock: {row['stock']}, Price: ${row['price']}, Season: {row.get('seasons', 'N/A')}",
    axis=1
)


# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))


,product_id,name,category,description,stock,seasons,price,text
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99,Retro Sunglasses. Accessories. Transport yours...
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00,Sleek Wallet. Accessories. Keep your essential...
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00,Versatile Scarf. Accessories. Add a touch of v...


,email_id,subject,body
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Embedding + Vector Store Setup

Generates semantic embeddings of product descriptions and initializes FAISS for vector-based product retrieval.



In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Create embedding model
embedding = OpenAIEmbeddings()

# Prepare texts and metadata
texts = products_df['text'].tolist()
metadata = products_df[["name", "price", "stock", "category", "description", "seasons"]].to_dict(orient="records")

# Create vector store with metadata attached
vector_store = FAISS.from_texts(texts=texts, embedding=embedding, metadatas=metadata)

# Initialize retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


# Processing Emails in Bulk

Iterates through all emails:

Classifies each email.

Handles orders or inquiries.

Stores responses for final export.

In [ ]:
# Initialize storage for results
order_results = []
order_responses = []
inquiry_responses = []

# Process each email
for _, row in emails_df.iterrows():
    email_id = row['email_id']
    subject = row['subject']
    body = row['body']

    print(f"\nProcessing email: {email_id}")

    # Step 1: Classify the email
    classification = classify_email(subject, body).strip().lower()
    emails_df.loc[_, 'classification'] = classification

    print(f"Classified as: {classification}")

    if classification == "order request":
        # Step 2: Process the order using LLM-based extraction
        order_output, response_output = process_order(email_id, body)

        # ✅ Extend lists to store all order details and responses
        order_results.extend(order_output)
        order_responses.extend(response_output)

    elif classification == "product inquiry":
        # Step 3: Handle inquiry using RAG
        response = handle_inquiry(email_id, body)
        inquiry_responses.append({
            "email_id": email_id,
            "response": response
        })
        print(f"Inquiry handled for {email_id}")

# Step 4: Save all results to Excel
save_to_excel()



Processing email: E001
Classified as: order request
Order processed for E001: Leather Bifold Wallet - created

Processing email: E002
Classified as: product inquiry
Handling inquiry for E002
Inquiry handled for E002

Processing email: E003
Classified as: product inquiry
Handling inquiry for E003
Inquiry handled for E003

Processing email: E004
Classified as: order request
Order processed for E004: Infinity Scarf - created

Processing email: E005
Classified as: product inquiry
Handling inquiry for E005
Inquiry handled for E005

Processing email: E006
Classified as: product inquiry
Handling inquiry for E006
Inquiry handled for E006

Processing email: E007
Classified as: order request
Order processed for E007: Cable Knit Beanie - out of stock
Order processed for E007: Fuzzy Slippers - created

Processing email: E008
Classified as: order request
Order processed for E008: Versatile Scarf - created

Processing email: E009
Classified as: product inquiry
Handling inquiry for E009
Inquiry hand